In [2]:
# if a module import does not work, use below
%pip install requests
%pip install pandas

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


You should consider upgrading via the 'c:\Program Files (x86)\Microsoft Visual Studio\Shared\Python39_64\python.exe -m pip install --upgrade pip' command.


# CSE587 Project

Saagnik Sarbadhikari, 50592327

Marcus Hartman, 50398874

Bharath Reddy, 50563984

# Forecasting the Rental Housing Economy of Various Areas Using Machine Learning

Data Retrieval

In [3]:
import json
import requests
import pandas

In [4]:
# saag

In [130]:
# Marcus' data

# huduser.gov
huduser_key = open('tokens/huduser').readline()

huduser_MN = requests.get("https://www.huduser.gov/hudapi/public/fmr/statedata/MN", headers={"Authorization": f"Bearer {huduser_key}"}).json()

huduser_TX = requests.get("https://www.huduser.gov/hudapi/public/fmr/statedata/TX", headers={"Authorization": f"Bearer {huduser_key}"}).json()

# zillow.com
zillow_rentals = pandas.read_csv("https://files.zillowstatic.com/research/public_csvs/zori/Metro_zori_uc_sfrcondomfr_sm_month.csv?t=1728278819")
# this is home values forecasts, not rentals. but still poses significance 
zillow_forecasts = pandas.read_csv("https://files.zillowstatic.com/research/public_csvs/zhvf_growth/Metro_zhvf_growth_uc_sfrcondo_tier_0.33_0.67_sm_sa_month.csv?t=1728278819")


# rentcast.io
# NOTE: any request using the API key will accumulate towards the monthly limit (31). please do not go over this

# the commented out part uses the API key you put into tokens/rentcast (req. credit card info). 
# the service is free but still requires credit card info-- if you find something better use it. update latex and make a little note next to it and we're all good
# rentcast_key = open('tokens/rentcast').readline()

# rentcast_TX = requests.get("https://api.rentcast.io/v1/listings/rental/long-term?state=TX", headers={"accept": "application/json", "X-Api-Key": rentcast_key})
# f = open("results/rentcast_TX", "w")
# f.write(json.dumps(rentcast_TX.json()))
# f.close()

# rentcast_MN = requests.get("https://api.rentcast.io/v1/listings/rental/long-term?state=MN", headers={"accept": "application/json", "X-Api-Key": rentcast_key})
# f = open("results/rentcast_MN", "w")
# f.write(json.dumps(rentcast_MN.json()))
# f.close()

# loads data stored from api calls above. costs money if we go above 31 calls / month
rentcast_TX_json = json.loads(open('results/rentcast_TX').readline())
rentcast_MN_json = json.loads(open('results/rentcast_MN').readline())

# airbnb
airbnb_TX_data = pandas.read_csv("https://data.insideairbnb.com/united-states/tx/dallas/2024-08-17/visualisations/listings.csv")
airbnb_MN_data = pandas.read_csv("https://data.insideairbnb.com/united-states/mn/twin-cities-msa/2024-06-24/visualisations/listings.csv")


In [93]:
# bharath

Data Cleaning

In [137]:
# need 10 different cleaning operations for all of our sources

# 1. cleaning hudusers.gov of unnecessary columns
def cleanCounties(countyInfo):
    columnsToDrop = ['town_name', 'metro_name', 'fips_code', 'FMR Percentile', 'metro_name', 'statename', 'statecode', 'smallarea_status']
    
    for county in countyInfo:
        for n in columnsToDrop:
            if n in county:
                county.pop(n)
        
    return countyInfo


countyInfo_MN = cleanCounties(huduser_MN['data']['counties'])
countyInfo_TX = cleanCounties(huduser_MN['data']['counties'])

#2. Zillow rentals: drop unnecessary columns
zillow_rentals.drop(["RegionID", "SizeRank", "RegionType"], axis=1, inplace=True, errors='ignore')

#3. Zillow rentals: drop all that aren't for MN, TX
zillowRentals_MN = zillow_rentals.query('RegionName.str.contains("MN")')
zillowRentals_TX = zillow_rentals.query('RegionName.str.contains("TX")')

#4. Zillow forecasts: drop unnecessary columns
zillow_forecasts.drop(["RegionID", "SizeRank", "RegionType", "BaseDate"], axis=1, inplace=True, errors='ignore')

#5. Zillow forecasts: drop all that aren't for MN, TX
zillowForecasts_MN = zillow_forecasts.query('StateName == "MN"')
zillowForecasts_TX = zillow_forecasts.query('StateName == "TX"')

#6. rentCast: drop unnecessary columns
def cleanRentals(rentalInfo):
    columnsToDrop = ['id', 'formattedAddress', 'addressLine1', 'addressLine2', 'latitude', 'longitude', 'status', 'listingType', 'listedDate', 'removedDate', 'createdDate', 'lastSeenDate', 'daysOnMarket', 'yearBuilt', 'history']
    
    for rental in rentalInfo:
        for c in columnsToDrop:
            if c in rental:
                rental.pop(c)

try:
    cleanRentals(rentcast_MN_json)
    cleanRentals(rentcast_TX_json)
except TypeError: # cell already ran
    pass

#7. rentCast: sort data by county, store in dictionary for easier lookup
def convertToDict(rentalInfo):
    res = dict()
    for rental in rentalInfo:
        county = rental['county']
        if county not in res:
            res[county] = []
        res[county].append(rental)
    return res

# only found in Hennepin, but lots to work with
rentcast_MN = convertToDict(rentcast_MN_json)
# only found in Dallas, but lots to work with
rentcast_TX = convertToDict(rentcast_TX_json)

#8. airbnb: get rid of unnecessary columns
airbnb_MN_data.drop(["id", "name", "host_name", "host_id", "neighbourhood_group", "latitude", "longitude", "last_review", "reviews_per_month", "number_of_reviews_ltm", "license"], axis=1, inplace=True, errors='ignore')
airbnb_TX_data.drop(["id", "name", "host_name", "host_id", "neighbourhood_group", "latitude", "longitude", "last_review", "reviews_per_month", "number_of_reviews_ltm", "license"], axis=1, inplace=True, errors='ignore')

#9. airbnb: get rid of duplicates
airbnb_MN_data.drop_duplicates()
airbnb_TX_data.drop_duplicates()

#9. airbnb: get rid of rows where price is not available
airbnb_MN_data.dropna(subset=['price'], how='all', inplace=True)
airbnb_TX_data.dropna(subset=['price'], how='all', inplace=True)

#10. airbnb: sort by room_type, convert to dictionary
def convertToDict_BNB(df : pandas.DataFrame):
    res = dict()
    for _, row in df.iterrows():
        roomType = row['room_type']
        if roomType not in res:
            res[roomType] = []
        res[roomType].append(row)
    return res
    
airbnb_MN = convertToDict_BNB(airbnb_MN_data)
airbnb_TX = convertToDict_BNB(airbnb_TX_data)


EDA

In [8]:
# saag

In [9]:
# marcus

In [10]:
# bharath